In [1]:
import time
import csv
import math
import re
from datetime import datetime
import numpy as np
import pandas as pd

+ '영화명', '개봉일', '매출액', '누적매출액', '관객수', '누적관객수',
    '스크린수', '상영횟수', '대표국적',  '배급사', '등급', '장르', '감독', '배우'
    
 df : 원본파일
 db : 배우감독배급사 비교 할 파일?
    
+ 개봉일
    - 1) nan 값에 대해서는 1000-01-01으로 변환.
    - 2) 연도 형식으로 통일(연도만 있는 것, 형식 다른 것 => 0000-00-00 형식으로 변환)
        - 수기 수정 가능
    - 3) datetime 데이터 타입으로 변환
    
 + 누적 관객수 / 스크린수 / 상영횟수
     -> int 값이니 변경 X
     
 
+ 대표국적 11개 one-hot

+ 장르  25개 one-hot

+ 등급 위계가 있는 숫자로 변환
    - 선호도 가장 높은 등급 기준으로 위계가 있는 숫자로 변환
        - 통계 분석을 통해 등급별 매출 순위 (15/12/전체/청불/등급없음)
            - df.groupby(["등급"]).sum().sort_values(by="누적관객수")
            
 
+ 배급사 / 감독 / 배우
    - 
            

 
 
 <!--  + 변수 삭제
     - 영화명 / 매출액 / 관객수 -->

# labeling 1

In [134]:
df = pd.read_csv('중간저장3.csv', encoding = 'utf-8')

In [136]:
df["개봉일"].sort_values().head(50)

5624                       1957-03-19 
10940                            1959 
20464                            1959 
1777                             1961 
2325                             1962 
10952                            1962 
11896                            1962 
1463                             1962 
10947                            1963 
8078                             1963 
10944                            1967 
211                           1967-01 
2250                             1968 
16331                            1968 
8236                             1969 
1803                             1969 
1489                             1970 
2246                             1970 
14240                      1975-05-24 
1797                             1977 
5572                             1986 
12155                         1989-12 
7552                       2004-08-06 
5631                       2006-05-12 
14080                      2006-10-27 
5266                     

In [21]:
pd.options.display.float_format = '{:.5f}'.format
#pd.reset_option('display.float_format')

### 개봉일 nan 값을 1000-01-01로 변환

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21325 entries, 0 to 21324
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영화명     21325 non-null  object
 1   개봉일     13946 non-null  object
 2   매출액     21325 non-null  int64 
 3   누적매출액   21325 non-null  int64 
 4   관객수     21325 non-null  int64 
 5   누적관객수   21325 non-null  int64 
 6   스크린수    21325 non-null  int64 
 7   상영횟수    21325 non-null  int64 
 8   대표국적    21325 non-null  object
 9   배급사     21325 non-null  object
 10  등급      21325 non-null  object
 11  장르      21325 non-null  object
 12  감독      21325 non-null  object
 13  배우      21325 non-null  object
dtypes: int64(6), object(8)
memory usage: 2.3+ MB


In [138]:
df['개봉일'].fillna("1900-01-01", inplace = True)

In [139]:
df[df['개봉일'] == "1900-01-01"]

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",1900-01-01,2192500,2192500,437,437,3,17,한국,0,등급없음,다큐멘터리,0,['0']
4,제3의앵글 : 형형색색대만전,1900-01-01,1830000,1830000,305,305,1,2,기타,0,등급없음,장르없음,0,['0']
9,페이스트리의 왕,1900-01-01,642000,642000,107,107,1,2,미국,0,등급없음,장르없음,0,['0']
19,동쪽 저 멀리,1900-01-01,160000,160000,31,31,1,3,미국,0,등급없음,장르없음,0,['0']
21,뱀파이어의 그림자,1900-01-01,101500,101500,15,15,1,2,기타,0,청소년관람불가,공포,E.엘리아스메리지,"['존말코비치', '윌렘대포', '캐리엘위스']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21316,포르투갈 여인,1900-01-01,205000,205000,32,32,1,2,기타,0,등급없음,드라마,0,"['마르셀로우르게게', '잉그리드카벤', '리타듀라오']"
21319,인터넷 중독자 수용소,1900-01-01,10000,10000,2,2,1,1,기타,0,등급없음,장르없음,힐라메달리아,['0']
21320,댄싱 인 자파,1900-01-01,88000,88000,15,15,1,2,미국,0,등급없음,다큐멘터리,힐라메달리아,['0']
21321,위기의 30대 여자들,1900-01-01,946400,946400,136,136,1,2,기타,0,등급없음,다큐멘터리,힐라메달리아,['0']


In [140]:
# 날짜 형식이 0000-00-00이 아닌 행의 인덱스 가져오기

index=[]
for i in range(len(df["개봉일"])):
    p= re.compile("\d\d\d\d-\d\d-\d\d")
    m= bool(p.match(str(df["개봉일"][i])))
    if m == False:
            index.append(i)

In [141]:
len(index)

78

In [143]:
pd.set_option('display.max_rows',3000)
df.iloc[index]["개봉일"].head(78)

185                            2008-04
186                            2008-04
187                            2008-04
188                            2008-04
211                           1967-01 
277                               1961
1207                              1969
1208                              1970
1463                             1962 
1489                             1970 
1775                              1965
1776                              1964
1777                             1961 
1778                              1966
1780                              1964
1797                             1977 
1803                             1969 
2111                       2014-12-31 
2237                              1963
2241                              1967
2246                             1970 
2250                             1968 
2251                              1961
2278                              1968
2325                             1962 
3436                     

In [144]:
# 개봉일 공백 제거
for i in range(len(df["개봉일"])):
    if type(df["개봉일"][i]) == float:
        df["개봉일"][i] = 0
    else:
        List = df["개봉일"][i].split(" ")
        df["개봉일"][i] = "".join(List)

<ipython-input-144-83c61f75f097>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["개봉일"][i] = "".join(List)


In [145]:
# 위에 거 확인 코드
# df.iloc[7262]["개봉일"][-10:]

'1952-10-20'

In [146]:

for j in range(len(df['개봉일'])):
    if len(df.iloc[j]['개봉일']) == 4:
        df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01-01"

    elif len(df.iloc[j]['개봉일']) == 7:
        df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01"
        
        
    elif  len(df.iloc[j]['개봉일']) >= 11:
        df['개봉일'][j] = df.iloc[j]['개봉일'][-10:]
        
    else: 
        continue

<ipython-input-146-d5ad4e9d3588>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01"
<ipython-input-146-d5ad4e9d3588>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01-01"
<ipython-input-146-d5ad4e9d3588>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['개봉일'][j] = df.iloc[j]['개봉일'][-10:]


In [147]:
df.iloc[index]["개봉일"].head(78) 

185      2008-04-01
186      2008-04-01
187      2008-04-01
188      2008-04-01
211      1967-01-01
277      1961-01-01
1207     1969-01-01
1208     1970-01-01
1463     1962-01-01
1489     1970-01-01
1775     1965-01-01
1776     1964-01-01
1777     1961-01-01
1778     1966-01-01
1780     1964-01-01
1797     1977-01-01
1803     1969-01-01
2111     2014-12-31
2237     1963-01-01
2241     1967-01-01
2246     1970-01-01
2250     1968-01-01
2251     1961-01-01
2278     1968-01-01
2325     1962-01-01
3436     1969-01-01
4549     2004-12-01
5266     2007-07-26
5572     1986-01-01
5624     1957-03-19
5631     2006-05-12
7261     1949-05-27
7262     1952-10-20
7552     2004-08-06
8078     1963-01-01
8236     1969-01-01
8258     1970-01-01
9703     1981-01-01
9807     2021-01-01
10027    2006-10-01
10525    1962-01-01
10929    1970-01-01
10930    1959-01-01
10931    1963-01-01
10933    1959-01-01
10935    1969-01-01
10936    1963-01-01
10938    1969-01-01
10940    1959-01-01
10944    1967-01-01


In [148]:
# datetime 데이터 타입으로 변환
df["개봉일"]=  pd.to_datetime(df["개봉일"], format="%Y-%m-%d")

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21325 entries, 0 to 21324
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   영화명     21325 non-null  object        
 1   개봉일     21325 non-null  datetime64[ns]
 2   매출액     21325 non-null  int64         
 3   누적매출액   21325 non-null  int64         
 4   관객수     21325 non-null  int64         
 5   누적관객수   21325 non-null  int64         
 6   스크린수    21325 non-null  int64         
 7   상영횟수    21325 non-null  int64         
 8   대표국적    21325 non-null  object        
 9   배급사     21325 non-null  object        
 10  등급      21325 non-null  object        
 11  장르      21325 non-null  object        
 12  감독      21325 non-null  object        
 13  배우      21325 non-null  object        
dtypes: datetime64[ns](1), int64(6), object(7)
memory usage: 2.3+ MB


In [151]:
df.to_csv("중간저장4.csv", encoding = 'utf-8-sig', index = False)

In [150]:
df["배우"]

0                                    ['0']
1               ['애덤존슨', '로버트제인', '케빈소르보']
2            ['모건브래들리', '그레이엄빈센트', '캠벨루셀']
3                             ['빌오버스트주니어']
4                                    ['0']
                       ...                
21320                                ['0']
21321                                ['0']
21322                 ['제임스캘로웨이', '스칼렛마셜']
21323                                ['0']
21324    ['제임스와일더', '조아큄드알메이다', '레베카메이파머']
Name: 배우, Length: 21325, dtype: object

2가지 경로
df 저희 돌리는 데이터
db 관객수 가져오는 데이터

df for 1~마지막행까지 배우1 2 3 
    for 배우 1,2,3
        배우1 db 배우 어디있는지 인덱스 가져오기
        db["인덱스"]  if 배우
        for db 날짜인덱스

for1 -> 배우 3명 분리

for2 배우1
    3년 데이터 할당

for3 배우2

for4 배우3


for len(df)
    def for1(df[배우][i])
    -> return 배우1, 배우2, 배우3
    def  3년 데이터 할당
    배우1 이 전체 db에 대해 배우 열에 있는 행의 인덱스
    이 영화 개봉일 기준 3년 내 개봉한 영화인지 개봉일 행의 인덱스
    db[공통인덱스.groupby("누적관객수"].sum()
    
    
    

In [ ]:
for num in range(len(df2["배우"])):
    if df2["배우"][num].strip("[]").split(",")[0].strip("'") =='0':
        df2.loc[num,"배우"] = 0
        
    else:
        if pd.isna(df2.iloc[i]["개봉일"]):
            df2.loc[i,["배급사","배우","감독"]] = 0
            
        else:
            date_tf = (df2.iloc[num]["개봉일"] > db2["개봉일"]) & (df2.iloc[num]["개봉일"]- relativedelta(years=3) < db2["개봉일"])
            act_list = df2["배우"][num].strip("[]").split(",")
            
            for act in act_list:
                for 
            
            
            
                
                act.strip("'")

In [ ]:
            
            for act in act_list:
                audi = 0
                count = 0
                if act.strip("'")== '0':
                       df2.loc[i,"배우"] = 0
                       
                else:
               for i in range(len(df2)):
    if i in index:
        
    else:
        act_list = df2.iloc[i]["배우"].strip("[]").split(",")
        

                   
                       act.strip("'") in db2["배우"][j]:
                       audi += db2["누적관객수"][j]
                       count += 1
            audi = audi /count

In [649]:
date_tf = (df2.iloc[1]["개봉일"] > db2["개봉일"]) & (df2.iloc[1]["개봉일"]- relativedelta(years=3) < db2["개봉일"])